<font size = "6"> Fordham Sports Analytics Society Big Data Bowl 2023 - Exploratory Analysis </font>

<font size = "4"> Run through ideas for eventual model, create new model features, and understand provided materials. </font>

- Authors:  Peter Majors, Chris Orlando, and Jack Townsend
- Kaggle:  https://www.kaggle.com/competitions/nfl-big-data-bowl-2023/overview (Resources)
- Our Github:  https://github.com/peterlmajors/FSAS_Big_Data_Bowl_2023 (Up-To-Date Code)

In [1]:
#Import Required Packages

#Data Manipulation
import pandas as pd
import numpy as np
import math

#Data Vizualization
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly.graph_objects as go

#Notebook Settings
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
#Importing Kaggle Data (Needed For Animated Plays Function (Takes Original Data))

# #Games - Basic Information On All Games
games = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/games.csv")

# #pffScout - PFF Judgements For Each Player On Each Play
pffScout = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/pffScoutingData.csv")

# #Players - Basics On Players
players = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/players.csv")

# #Plays - Everthing About Specific Plays
plays = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/plays.csv")

# #Week - Frame-By-Frame Player Tracking
week1 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/week1.csv")
week2 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/week2.csv")
week3 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/week3.csv")
week4 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/week4.csv")
week5 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/week5.csv")
week6 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/week6.csv")
week7 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/week7.csv")
week8 = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/case_data/week8.csv")

In [3]:
#Import Our Merged DataFrames (Needed)
ptrack = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/merged_data/ptrack.csv")
pbp = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/merged_data/pbp.csv")

C:\Users\Peter\AppData\Local\Temp\ipykernel_9868\1133389364.py:2: DtypeWarning: Columns (14,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  ptrack = pd.read_csv("C:/Users/Peter/Python Scripts/Case Competitions/NFL Big Data Bowl 2023/FSAS_Big_Data_Bowl_2023/merged_data/ptrack.csv")


In [4]:
#Define Function To Animate Plays (Needed)
colors = {'ARI':"#97233F", 'ATL':"#A71930", 'BAL':'#241773', 'BUF':"#00338D", 'CAR':"#0085CA", 'CHI':"#C83803", 'CIN':"#FB4F14", 'CLE':"#311D00", 
          'DAL':'#003594', 'DEN':"#FB4F14", 'DET':"#0076B6", 'GB':"#203731", 'HOU':"#03202F", 'IND':"#002C5F", 'JAX':"#9F792C", 'KC':"#E31837", 
          'LA':"#003594", 'LAC':"#0080C6", 'LV':"#000000", 'MIA':"#008E97", 'MIN':"#4F2683", 'NE':"#002244", 'NO':"#D3BC8D", 'NYG':"#0B2265", 
          'NYJ':"#125740", 'PHI':"#004C54", 'PIT':"#FFB612", 'SEA':"#69BE28", 'SF':"#AA0000", 'TB':'#D50A0A', 'TEN':"#4B92DB", 'WAS':"#5A1414", 
          'football':'#CBB67C'}

#TODO:
# Add Useful Player Labels
# Add ability to keep or remove the extra stuff (line markers etc)
# Add Team Colors to dict
# Turn into Function 

def animate_play(tracking_df, play_df,players,pffScoutingData, gameId,playId):
    selected_play_df = play_df[(play_df.playId==playId)&(play_df.gameId==gameId)].copy()
    
    tracking_players_df = pd.merge(tracking_df,players,how="left",on = "nflId")
    tracking_players_df = pd.merge(tracking_players_df,pffScoutingData,how="left",on = ["nflId","playId","gameId"])
    selected_tracking_df = tracking_players_df[(tracking_players_df.playId==playId)&(tracking_players_df.gameId==gameId)].copy()

    sorted_frame_list = selected_tracking_df.frameId.unique()
    sorted_frame_list.sort()

    # get play General information 
    line_of_scrimmage = selected_play_df.absoluteYardlineNumber.values[0]
    first_down_marker = line_of_scrimmage + selected_play_df.yardsToGo.values[0]
    down = selected_play_df.down.values[0]
    quarter = selected_play_df.quarter.values[0]
    gameClock = selected_play_df.gameClock.values[0]
    playDescription = selected_play_df.playDescription.values[0]
    # Handle case where we have a really long Play Description and want to split it into two lines
    if len(playDescription.split(" "))>15 and len(playDescription)>115:
        playDescription = " ".join(playDescription.split(" ")[0:16]) + "<br>" + " ".join(playDescription.split(" ")[16:])

    # initialize plotly start and stop buttons for animation
    updatemenus_dict = [
        {
             "buttons": [
                {
                    "args": [None, {"frame": {"duration": 100, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 0}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]
    # initialize plotly slider to show frame position in animation
    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }


    frames = []
    for frameId in sorted_frame_list:
        data = []
        # Add Numbers to Field 
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        data.append(
            go.Scatter(
                x=np.arange(20,110,10), 
                y=[53.5-5]*len(np.arange(20,110,10)),
                mode='text',
                text=list(map(str,list(np.arange(20, 61, 10)-10)+list(np.arange(40, 9, -10)))),
                textfont_size = 30,
                textfont_family = "Courier New, monospace",
                textfont_color = "#ffffff",
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add line of scrimage 
        data.append(
            go.Scatter(
                x=[line_of_scrimmage,line_of_scrimmage], 
                y=[0,53.5],
                line_dash='dash',
                line_color='blue',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Add First down line 
        data.append(
            go.Scatter(
                x=[first_down_marker,first_down_marker], 
                y=[0,53.5],
                line_dash='dash',
                line_color='yellow',
                showlegend=False,
                hoverinfo='none'
            )
        )
        # Plot Players
        for team in selected_tracking_df.team.unique():
            plot_df = selected_tracking_df[(selected_tracking_df.team==team)&(selected_tracking_df.frameId==frameId)].copy()
            if team != "football":
                hover_text_array=[]
                for nflId in plot_df.nflId:
                    selected_player_df = plot_df[plot_df.nflId==nflId]
                    hover_text_array.append("nflId:{}<br>displayName:{}<br>Position:{}<br>Role:{}".format(selected_player_df["nflId"].values[0],
                                                                                      selected_player_df["displayName"].values[0],
                                                                                      selected_player_df["pff_positionLinedUp"].values[0],
                                                                                      selected_player_df["pff_role"].values[0]))
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team,hovertext=hover_text_array,hoverinfo="text"))
            else:
                data.append(go.Scatter(x=plot_df["x"], y=plot_df["y"],mode = 'markers',marker_color=colors[team],name=team,hoverinfo='none'))
        # add frame to slider
        slider_step = {"args": [
            [frameId],
            {"frame": {"duration": 100, "redraw": False},
             "mode": "immediate",
             "transition": {"duration": 0}}
        ],
            "label": str(frameId),
            "method": "animate"}
        sliders_dict["steps"].append(slider_step)
        frames.append(go.Frame(data=data, name=str(frameId)))

    scale=10
    layout = go.Layout(
        autosize=False,
        width=120*scale,
        height=60*scale,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array',tickvals=np.arange(10, 111, 5).tolist(),showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False,showgrid=False,showticklabels=False),

        plot_bgcolor='#00B140',
        # Create title and add play description at the bottom of the chart for better visual appeal
        title=f"GameId: {gameId}, PlayId: {playId}<br>{gameClock} {quarter}Q"+"<br>"*19+f"{playDescription}",
        updatemenus=updatemenus_dict,
        sliders = [sliders_dict]
    )

    fig = go.Figure(
        data=frames[0]["data"],
        layout= layout,
        frames=frames[1:]
    )
    # Create First Down Markers 
    for y_val in [0,53]:
        fig.add_annotation(
                x=first_down_marker,
                y=y_val,
                text=str(down),
                showarrow=False,
                font=dict(
                    family="Courier New, monospace",
                    size=16,
                    color="black"
                    ),
                align="center",
                bordercolor="black",
                borderwidth=2,
                borderpad=4,
                bgcolor="#ff7f0e",
                opacity=1
                )

    return fig

<font size="5">Check Out Individual Plays</font>

Picking A Passing Play From This Game

https://nfl-video.com/archive/nfl_replay/nfl_2020/green_bay_packers_vs_tampa_bay_buccaneers_full_game_highlights_nfl_2020_week_6/5-1-0-305

In [ ]:
#Run Animate Plays Function

#Choose A Game and Play
gameId = 2021090900
playId =  137

#Print The Play!=
animate_play(week1, plays, players, pffScout, gameId, playId)

In [ ]:
#Print Out All Frames From Selected Play
ptrack_filtered = np.where((ptrack['gameId'] == gameId) & (ptrack['playId'] == playId))
display(ptrack.loc[ptrack_filtered].head())

#Print Out Single Frame From Selected Play
#frame = 1
#ptrack_filtered = np.where((ptrack['gameId'] == gameId) & (ptrack['playId'] == playId) & (ptrack['frameId'] == 1))
#display(ptrack.loc[ptrack_filtered].head())

#Print Out All pbp Data From Selected Play
#pbp_filtered = np.where((pbp['gameId'] == gameId) & (pbp['playId'] == playId))
#display(pbp.loc[pbp_filtered].head())

# Print Out All pffScout Data From Selected Play
# pffScout_filtered = np.where((pffScout['gameId'] == gameId) & (pffScout['playId'] == playId))
# display(pffScout.loc[pffScout_filtered])

<font size="5">Explore Effect Of Block Types On Hurry Rate</font>

In [5]:
#For Each Block Type, Find The Success Rate of A Defesnvie Player Creating A Pressure

# #How We Found Positions
# #pffScout.pff_positionLinedUp.sort_values().unique()

# #Create A New Data Frame Only With 9 D-Linemen Positons
# ptrackl = ptrack.loc[(ptrack['pff_positionLinedUp'] == "LOLB") | (ptrack['pff_positionLinedUp'] == "ROLB") | (ptrack['pff_positionLinedUp'] == "LEO") |
#     (ptrack['pff_positionLinedUp'] == "REO") | (ptrack['pff_positionLinedUp'] == "DLT") | (ptrack['pff_positionLinedUp'] == "DRT") |
#     (ptrack['pff_positionLinedUp'] == "NLT") | (ptrack['pff_positionLinedUp'] == "NRT") | (ptrack['pff_positionLinedUp'] == "NT")]

# #Must First Change The pff_blockType in ptrack To A String (Data Did Not Group By Properly When An Int)
# ptrackl.pff_blockType = ptrack.pff_blockType.astype("str")

# #For Each Block Type, Find The Percentage of Time A Hury Occurs
# ptrackl.groupby('pff_blockType')['pff_hurry'].count() #Ideally Would Be .average() Instead

# #Frequency of Different Type of Blocks Performed (EDA)
# #ptrack.pff_blockType.value_counts()

# #Issue Is Only The Players Issuing The Block (Offensive Players) Are Assigned A Block Type
# #However, The Defensive Players They Blocked Are Noted In The pff_nflIdBlockedPlayer Column In ptrack

C:\Users\Peter\AppData\Local\Temp\ipykernel_9868\3845793339.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptrackl.pff_blockType = ptrack.pff_blockType.astype("str")


pff_blockType
0    1315558
Name: pff_hurry, dtype: int64

In [66]:
# #Create A Data Frame Containing Only Plays Where A Pass Block Occured (One Row Per Block)
# ptrack_blocks = ptrack.loc[ptrack['pff_blockType'] != '0'][['gameId', 'playId', 'nflId', 'pff_nflIdBlockedPlayer', 'pff_blockType', 'pff_hurryAllowed']]

# #Reduce To Each Play (One Row Per Interaction Between Blocked Player And Blocker) (First Interation of A Defensive Player Being Blocked In Play)
# ptrack_blocks = ptrack_blocks.drop_duplicates() 

# #Sampler of The ptrack_blocks Data Frame
# ptrack_blocks.sort_values('gameId', ascending = False).head(5)

# #Show How Often Each Block Type Resulted In A Hurry
# ptrack_block_hurry = pd.DataFrame(ptrack_blocks.groupby('pff_blockType')['pff_hurryAllowed'].mean().round(4).sort_values(ascending = False))
# ptrack_block_hurry

,pff_hurryAllowed
pff_blockType,
PU,0.1061
SW,0.0658
UP,0.0584
PP,0.0574
PA,0.0306
BH,0.0248
PT,0.0176
PR,0.0136
SR,0.0100


In [67]:
# #Can Each Offensive Player Only Get One Block Per Play (I'd Assume Not, But Lets Check) (Yes They Do!)
# ptrack_blocks_per_play = pd.DataFrame(ptrack_blocks.groupby(['gameId', 'playId', 'nflId'])['pff_blockType'].count())
# ptrack_blocks_per_play = ptrack_blocks_per_play.reset_index()

# ptrack_blocks_per_play[ptrack_blocks_per_play['pff_blockType'] != 1].head() #No Rows Appear
# #Yes! Each Offensive Player Only Gets One Block Per Play!

,gameId,playId,nflId,pff_blockType


<font size="5"> Explore Effect Of Distance From Quarterback At Each Second </font>

Each D-Line Position and Each Second Post Snap